# Medlytics Final Project Notebook

## Importing Libraries

In [1]:
! /root/venv/bin/python -m pip install --upgrade pip
! pip3 uninstall googletrans -y
! pip3 install googletrans==3.1.0a0

Found existing installation: googletrans 3.1.0a0
Uninstalling googletrans-3.1.0a0:
  Successfully uninstalled googletrans-3.1.0a0
  Using cached googletrans-3.1.0a0-py3-none-any.whl
  Using cached idna-2.10-py2.py3-none-any.whl (58 kB)
  Attempting uninstall: idna
    Found existing installation: idna 3.3
    Not uninstalling idna at /shared-libs/python3.9/py-core/lib/python3.9/site-packages, outside environment /root/venv
    Can't uninstall 'idna'. No files were found to uninstall.


In [2]:
#! pip3 install googletrans

from googletrans import Translator, constants
from pprint import pprint

translator = Translator()

translation = translator.translate("讨厌的人呢，应该比较讨厌那种")
print(f"{translation.origin} ({translation.src}) --> {translation.text} ({translation.dest})")

讨厌的人呢，应该比较讨厌那种 (zh-CN) --> Those who hate it, should hate that more (en)


In [3]:
# Import basic system and numerical packages
import os
import re
!pip install numpy==1.22
import numpy as np
import pandas as pd

# Import statistical and signal processing functions
from scipy import signal
import scipy.stats.mstats as mstats
from sklearn import metrics

# Import the scikit-learn functions and models
from sklearn.model_selection import train_test_split
from sklearn import tree

# Import audio i/o and playing functions
import scipy.io.wavfile
from IPython.display import Audio

# Import plotting functionality
import matplotlib
import matplotlib.pyplot as plt
import graphviz
import seaborn as sns
sns.set()

#librosa
!sudo apt-get update && sudo apt-get install -y libsndfile1
!pip install librosa
import librosa

from pydub import AudioSegment
from scipy.io import wavfile

import tensorflow as tf
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences

import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')

from tensorflow.keras import Input, Model
from keras.layers import LSTM, Dense, Embedding, concatenate, Dropout, concatenate
from keras.layers import Bidirectional
from tensorflow.keras import regularizers

from sklearn.metrics import accuracy_score

Hit:1 http://security.debian.org/debian-security buster/updates InRelease
Hit:2 http://deb.debian.org/debian buster InRelease
Hit:3 http://deb.debian.org/debian buster-updates InRelease




libsndfile1 is already the newest version (1.0.28-6+deb10u1).
0 upgraded, 0 newly installed, 0 to remove and 15 not upgraded.
/root/venv/lib/python3.9/site-packages/pydub/utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Data Processing

In [13]:
#cols = ["text", "label"]
data = pd.DataFrame()
val_data = pd.DataFrame()

In [14]:
count = 0

#!git clone https://github.com/DavidWangBWSI/MedlyticsFinalProject.git

filenames = ['/negative.txt', '/neutral.txt', '/positive.txt']

for i in range(1,112):

    try:
        path = 'Training/t_'+str(i)

        with open(path+'/new_label.txt') as f:
            label = float(f.readline())
            label = int(label>=53)

        for name in filenames:
            with open(path+name) as f:
                text = f.readline()
                # TODO clean text
                data.loc[count, "text"] = translator.translate(text).text
                data.loc[count, 'label'] = label
                count+=1

    except:
        continue

In [15]:
data

,text,label
0,"As for the person you hate, you should hate th...",0.0
1,"In theory, it should be more hopeful to be abl...",0.0
2,"Well, when you are relaxing, you should like w...",0.0
3,I should have started learning English in elem...,0.0
4,I want to work on commuting to get off work on...,0.0
...,...,...
228,"I don't like to be in contact with them, it's ...",1.0
229,I heard the song of my favorite band during gy...,1.0
230,The thing I regret most is not having the abil...,1.0
231,"I don't like children. Just, I would think the...",1.0


In [16]:


#!git clone https://github.com/DavidWangBWSI/MedlyticsFinalProject.git

filenames = ['/negative.txt', '/neutral.txt', '/positive.txt']

for i in range(1,112):

    try:
        path = 'Validation/v_'+str(i)

        with open(path+'/new_label.txt') as f:
            label = float(f.readline())
            label = int(label>=53)

        for name in filenames:
            with open(path+name) as f:
                text = f.readline()
                # TODO clean text
                data.loc[count, "text"] = translator.translate(text).text
                data.loc[count, 'label'] = label
                count+=1

    except:
        continue

In [18]:
import re
import nltk
from sklearn.datasets import load_files
nltk.download('stopwords')
import pickle
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [19]:
import tensorflow as tf

columns = ['text']
X_data = data[columns].to_numpy()

y_labels = data['label'].to_numpy()

#convert labels to onehot
# y_labels_train = tf.keras.utils.to_categorical(y_labels_train, 2)
# y_labels_val = tf.keras.utils.to_categorical(y_labels_val, 2)

# training_set = tf.data.Dataset.from_tensor_slices((X_data_train, y_labels_train))

In [20]:
X_data[0]

array(["As for the person you hate, you should hate that kind of person, especially the self-centered, or, and then, he thinks he is right about everything, and then, he especially likes to argue with you, and I especially hate it. Then, um, or else, I don't really understand, I know, I also hate people like me in terms of how people interact with each other."],
      dtype=object)

In [21]:
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')

from nltk.stem import WordNetLemmatizer

documents = []

stemmer = WordNetLemmatizer()

for sen in range(0, len(X_data)):
    # Remove all the special characters

    document = re.sub(r'\W', ' ', str(X_data[sen]))
    
    # remove all single characters
    document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)
    
    # Remove single characters from the start
    document = re.sub(r'\^[a-zA-Z]\s+', ' ', document) 
    
    # Substituting multiple spaces with single space
    document = re.sub(r'\s+', ' ', document, flags=re.I)
    
    # Removing prefixed 'b'
    document = re.sub(r'^b\s+', '', document)
    
    # Converting to Lowercase
    document = document.lower()
    
    # Lemmatization
    document = document.split()

    document = [stemmer.lemmatize(word) for word in document]
    document = ' '.join(document)
    
    documents.append(document)


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [22]:
documents

['a for the person you hate you should hate that kind of person especially the self centered or and then he think he is right about everything and then he especially like to argue with you and especially hate it then um or else don really understand know also hate people like me in term of how people interact with each other',
 'in theory it should be more hopeful to be able to engage in the work that you like that is to have fun while working well ve thought about this myself for example because my major is soft engineering it is reasonable to say that should go to programming just type code well do really like this profession m not so sure either however at least it salary should still be',
 'well when you are relaxing you should like watching um watching movie because watching movie is relatively entertaining activity and it doesn require lot of brainstorming so hey ve been watching movie recently anyway',
 'should have started learning english in elementary school',
 'want to work 

In [23]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(max_features=1500, min_df=0, max_df=1, stop_words=stopwords.words('english'))
X = vectorizer.fit_transform(documents).toarray()

In [24]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidfconverter = TfidfTransformer()
X = tfidfconverter.fit_transform(X).toarray()

In [25]:
# TODO change parameters to allow for overfitting
# TODO check to make sure inputs r correct
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y_labels, test_size=0.15, random_state=0)

In [ ]:
# from sklearn.ensemble import RandomForestClassifier

# classifier = RandomForestClassifier(n_estimators=1000, random_state=0)
# classifier.fit(X_train, y_train)

# y_pred = classifier.predict(X_test)

In [26]:
# from sklearn.linear_model import LogisticRegression

# logreg = LogisticRegression(class_weight = 'auto')
# logreg.fit(X_train,y_train)

# y_pred = logreg.predict(X_test)

In [28]:
! pip install xgboost
from xgboost import XGBClassifier

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.9/192.9 MB 11.6 MB/s eta 0:00:00


In [29]:
learning_rate=0.0005
n_estimators=1000
max_depth=7
subsample=1
colsample_bytree=1
gamma=1

xgb = XGBClassifier(learning_rate=learning_rate, n_estimators=n_estimators, max_depth=max_depth, subsample=subsample, colsample_bytree=colsample_bytree, gamma=gamma)
xgb.fit(X_train,y_train)

y_pred = xgb.predict(X_test)

In [ ]:
# y_train = tf.keras.utils.to_categorical(y_train, 2)
# y_test = tf.keras.utils.to_categorical(y_test, 2)

# training_set = tf.data.Dataset.from_tensor_slices((X_train, y_train))

# # Build the keras neural network
# #this allows us to add layers sequentially (i.e. first->last)
# model = tf.keras.Sequential()

# #create a first layer of 12 neurons, and a rectified linear unit activation function
# model.add(tf.keras.layers.Dense(8, input_shape=(X_train.shape[1],), activation=tf.nn.relu)) #input dimension needs to be number of features

# #add two dense layers with 8 units each
# #(note that we don't need to specify input size because keras determines input size from previous layer)
# model.add(tf.keras.layers.Dense(8, activation=tf.nn.relu))
# model.add(tf.keras.layers.Dense(8, activation=tf.nn.relu))
# model.add(tf.keras.layers.Dense(8, activation=tf.nn.tanh))

# # output dimension needs to be number of classes in order for each to get a score
# model.add(tf.keras.layers.Dense(2, activation=tf.nn.softmax)) 

# model.compile(loss = tf.keras.losses.categorical_crossentropy,
#         optimizer = tf.keras.optimizers.Adam(learning_rate=0.002),
#         metrics = ['accuracy'])
# model.summary()


# epochs = 50
# batch_size = 100

# training_set = training_set.batch(batch_size) #set batch size

# for epoch in range(epochs):
#     for signals, labels in training_set:
#         tr_loss, tr_accuracy = model.train_on_batch(signals, labels)
#     val_loss, val_accuracy = model.evaluate(X_test, y_test)
#     print(('Epoch #%d\t Training Loss: %.2f\tTraining Accuracy: %.2f\t'
#          'Validation Loss: %.2f\tValidation Accuracy: %.2f')
#          % (epoch + 1, tr_loss, tr_accuracy,
#          val_loss, val_accuracy))

# y_pred = model.predict(X_test)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 8)                 7496      
                                                                 
 dense_1 (Dense)             (None, 8)                 72        
                                                                 
 dense_2 (Dense)             (None, 8)                 72        
                                                                 
 dense_3 (Dense)             (None, 8)                 72        
                                                                 
 dense_4 (Dense)             (None, 2)                 18        
                                                                 
Total params: 7,730
Trainable params: 7,730
Non-trainable params: 0
_________________________________________________________________


In [30]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test, y_pred))

[[54  0]
 [16  0]]
              precision    recall  f1-score   support

         0.0       0.77      1.00      0.87        54
         1.0       0.00      0.00      0.00        16

    accuracy                           0.77        70
   macro avg       0.39      0.50      0.44        70
weighted avg       0.60      0.77      0.67        70

0.7714285714285715
/shared-libs/python3.9/py/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/shared-libs/python3.9/py/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_sta

## Word Embedding

In [31]:
wedata = pd.DataFrame()
wedata['Text'] = data['text']
wedata['Label'] = data['label']

In [32]:
stop_words=['the', 'a', 'and', 'is', 'be', 'will']
def clean_text(string: str, punctuations=r'''!()-[]{};:'"\,<>./?@#$%^&*_~''', stop_words=['the', 'a', 'and', 'is', 'be', 'will']) -> str:
    """
    A method to clean text 
    """
    # Cleaning the urls
    string = re.sub(r'https?://\S+|www\.\S+', '', string)

    # Cleaning the html elements
    string = re.sub(r'<.*?>', '', string)

    # Removing the punctuations
    for x in string.lower(): 
        if x in punctuations: 
            string = string.replace(x, "") 

    # Converting the text to lower
    string = string.lower()

    # Removing stop words
    string = ' '.join([word for word in string.split() if word not in stop_words])

    # Cleaning the whitespaces
    string = re.sub(r'\s+', ' ', string).strip()

    return string 

In [33]:
class Embeddings():
    """
    A class to read the word embedding file and to create the word embedding matrix
    """

    def __init__(self, path, vector_dimension):
        self.path = path 
        self.vector_dimension = vector_dimension
    
    @staticmethod
    def get_coefs(word, *arr): 
        return word, np.asarray(arr, dtype='float32')

    def get_embedding_index(self):
        embeddings_index = dict(self.get_coefs(*o.split(" ")) for o in open(self.path, errors='ignore'))
        return embeddings_index

    def create_embedding_matrix(self, tokenizer, max_features):
        """
        A method to create the embedding matrix
        """
        model_embed = self.get_embedding_index()

        embedding_matrix = np.zeros((max_features + 1, self.vector_dimension))
        for word, index in tokenizer.word_index.items():
            if index > max_features:
                break
            else:
                try:
                    embedding_matrix[index] = model_embed[word]
                except:
                    continue
        return embedding_matrix

class TextToTensor():

    def __init__(self, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.max_len = max_len

    def string_to_tensor(self, string_list: list) -> list:
        """
        A method to convert a string list to a tensor for a deep learning model
        """    
        string_list = self.tokenizer.texts_to_sequences(string_list)
        string_list = pad_sequences(string_list, maxlen=self.max_len)
        
        return string_list

class RnnModel():
    """
    A recurrent neural network for semantic analysis
    """

    def __init__(self, embedding_matrix, embedding_dim, max_len, X_additional=None):
        
        inp1 = Input(shape=(max_len,))
        x = Embedding(embedding_matrix.shape[0], embedding_dim, weights=[embedding_matrix])(inp1)
        x = Bidirectional(LSTM(256, return_sequences=True, dropout=0.5, recurrent_dropout=0.3))(x)
        x = Bidirectional(LSTM(150, dropout=0.5, recurrent_dropout=0.3))(x)
        x = Dense(128, activation="relu", kernel_regularizer=regularizers.L2(l2=1e-4))(x)
        x = Dropout(.8)(x)
        x = Dense(64, activation="relu")(x)
        x = Dropout(.8)(x)
        x = Dense(1, activation="sigmoid")(x)    
        model = Model(inputs=inp1, outputs=x)

        model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = 'accuracy')
        self.model = model

class Pipeline:
    """
    A class for the machine learning pipeline
    """
    def __init__(
        self, 
        X_train: list, 
        Y_train: list, 
        embed_path: str, 
        embed_dim: int,
        stop_words=[],
        X_test=[], 
        Y_test=[],
        epochs=3,
        batch_size=256
        ):

        # Preprocecing the text
        X_train = [clean_text(text, stop_words=stop_words) for text in X_train]
        Y_train = np.asarray(Y_train)
        
        # Tokenizing the text
        tokenizer = Tokenizer()
        tokenizer.fit_on_texts(X_train)

        # Creating the embedding matrix
        embedding = Embeddings(embed_path, embed_dim)
        embedding_matrix = embedding.create_embedding_matrix(tokenizer, len(tokenizer.word_counts))

        # Creating the padded input for the deep learning model
        max_len = np.max([len(text.split()) for text in X_train])
        TextToTensor_instance = TextToTensor(
            tokenizer=tokenizer, 
            max_len=max_len
            )
        X_train = TextToTensor_instance.string_to_tensor(X_train)

        # Creating the model
        rnn = RnnModel(
            embedding_matrix=embedding_matrix, 
            embedding_dim=embed_dim, 
            max_len=max_len
        )
        rnn.model.fit(
            X_train,
            Y_train, 
            batch_size=batch_size, 
            epochs=epochs
        )

        self.model = rnn.model
        self.tokenizer = tokenizer
        self.max_len = max_len

        # If X_test is provided we make predictions with the created model
        if len(X_test)>0:
            X_test = [clean_text(text) for text in X_test]
            X_test = TextToTensor_instance.string_to_tensor(X_test)
            yhat = [x[0] for x in rnn.model.predict(X_test).tolist()]
            
            self.yhat = yhat

            # If true labels are provided we calculate the accuracy of the model
            if len(Y_test)>0:
                self.acc = accuracy_score(Y_test, [1 if x > 0.5 else 0 for x in yhat])
                print('ytest: '+str(len(Y_test)))
                print('yhat: '+str(len(yhat)))

In [34]:
X_train_we, X_test_we, Y_train_we, Y_test_we = train_test_split(wedata['Text'].tolist(),wedata['Label'].tolist(), test_size = .33, shuffle = True)

results = Pipeline(
X_train=X_train_we,
Y_train=Y_train_we,
embed_path='embeddings/glove.twitter.27B.100d.txt',
embed_dim=100,
stop_words=stop_words,
X_test=X_test_we,
Y_test=Y_test_we,
epochs=50,
batch_size=25
)

Epoch 1/50
13/13 [==============================] - 19s 811ms/step - loss: 0.7438 - accuracy: 0.4677
Epoch 2/50
13/13 [==============================] - 10s 775ms/step - loss: 0.6639 - accuracy: 0.6290
Epoch 3/50
13/13 [==============================] - 10s 769ms/step - loss: 0.6404 - accuracy: 0.7226
Epoch 4/50
13/13 [==============================] - 11s 819ms/step - loss: 0.5735 - accuracy: 0.7226
Epoch 5/50
13/13 [==============================] - 10s 772ms/step - loss: 0.5619 - accuracy: 0.7323
Epoch 6/50
13/13 [==============================] - 10s 766ms/step - loss: 0.5747 - accuracy: 0.7548
Epoch 7/50
13/13 [==============================] - 10s 755ms/step - loss: 0.5293 - accuracy: 0.7806
Epoch 8/50
13/13 [==============================] - 11s 827ms/step - loss: 0.5279 - accuracy: 0.7839
Epoch 9/50
13/13 [==============================] - 10s 766ms/step - loss: 0.5527 - accuracy: 0.7839
Epoch 10/50
13/13 [==============================] - 10s 762ms/step - loss: 0.5279 - accura

In [35]:
print(Y_test_we)

[0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]


In [36]:
y_pred = [1 if x > 0.5 else 0 for x in results.yhat]

In [37]:
confusion_matrix(Y_test_we,y_pred)

array([[100,  23],
       [ 23,   8]])

In [38]:
print(results.acc)

0.7012987012987013


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=311e6adc-3eb1-4202-832e-d715f09028ff' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>